In [1]:
import numpy as np
import polars as pl

In [2]:
df = pl.read_csv('inventory_mana_cleaned.csv')
print(df.shape)
df.head()

(8000, 14)


ordernumber,orderdate,year,month,day,customername,customer_type,warehousecode,product_code,orderquantity,unitprice,revenue,costs,profit
str,str,i64,i64,i64,str,str,str,str,i64,f64,f64,i64,f64
"""SO - 000101""","""2015-08-26""",2015,8,26,"""New Ltd""","""Distributor""","""NXH382""","""STK-112""",5,643.5,3217.5,1430,1787.5
"""SO - 000102""","""2016-01-13""",2016,1,13,"""Winthrop ""","""Wholesale""","""GUT930""","""STK-121""",5,346.48,1732.4,1220,512.4
"""SO - 000103""","""2015-09-19""",2015,9,19,"""Apollo Ltd""","""Export""","""GUT930""","""STK-112""",10,643.5,6435.0,2860,3575.0
"""SO - 000104""","""2015-12-10""",2015,12,10,"""Eminence Corp""","""Export""","""AXW291""","""STK-150""",6,217.14,1302.84,846,456.84
"""SO - 000105""","""2015-02-19""",2015,2,19,"""E. Ltd""","""Wholesale""","""AXW291""","""STK-158""",6,143.1,858.6,540,318.6


In [3]:
df_prod = \
df.pivot(
    values = 'orderquantity',
    index = ['year', 'month'],
    columns = 'product_code',
    aggregate_function = 'sum'
).sort(
    ['year', 'month'], descending = False
).fill_null(
    0
).select(
    df.select(pl.col('product_code').unique()).to_series().to_list()
)

df_prod = \
df_prod.std(
).select(
    df.select(pl.col('product_code').unique()).to_series().to_list()
).transpose(
    include_header = True
).rename({
    'column': 'product_code',
    'column_0': 'std'
}).join(
    df_prod.mean(
    ).select(
        df.select(pl.col('product_code').unique()).to_series().to_list()
    ).transpose(
        include_header = True
    ).rename({
        'column': 'product_code',
        'column_0': 'mean'
    }), on = 'product_code'
).with_columns(
    (pl.col('std')/pl.col('mean')).alias('cov')
).with_columns(
    pl.when(pl.col('cov') <= pl.quantile('cov', 0.25))
    .then('X')
    .otherwise(pl.when(pl.col('cov') > pl.quantile('cov', 0.75))
              .then('Z')
              .otherwise('Y')
              ).alias('xyz')
).join(
    df.groupby(
        'product_code'
    ).agg(
        pl.col('revenue').sum()
    ).with_columns(
        (pl.col('revenue')/pl.sum(df.get_column('revenue'))).alias('perc')
    ).sort(
        'perc', descending = False
    ).with_columns(
        pl.cumsum('perc').alias('perc_cumsum')
    ).with_columns(
        pl.when(pl.col('perc_cumsum') < 0.6)
        .then('C')
        .otherwise(pl.when(pl.col('perc_cumsum') >= 0.8)
                  .then('A')
                  .otherwise('B')
                  ).alias('abc')
    ), on = 'product_code'
).with_columns(
    (pl.col('abc') + pl.col('xyz')).alias('type')
)